In [1]:
import numpy as np 
from scipy.optimize import curve_fit
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from ipywidgets import interact
from thermo import Chemical, UNIFAC

In [2]:

conc = {'t': [0, 450, 850, 1256, 1664, 1969, 2343, 2751, 3159, 3600, 4381, 5026, 5875, 6928] , 
    'IB':  [1.85,1.519, 1.25, 1, 0.835, 0.675, 0.55, 0.47, 0.4, 0.345, 0.325, 0.325, 0.325, 0.325], 
    'EtOH': [1.8, 1.425, 1.175, 0.98, 0.79, 0.635, 0.5, 0.435, 0.350, 0.29, 0.275, 0.275, 0.275, 0.275], 
    'ETBE': [0, 0.325, 0.6, 0.81, 1, 1.15, 1.29, 1.375, 1.43, 1.475, 1.519, 1.519, 1.519, 1.519] }

def activity(T, P, Q, arr): # takes Fi (mol/s)
    x_ls = [C/sum(arr) for C in arr]
    UNIFAC_DG = fetch_UNIFAC_DG()
    return [g*x for g, x, in zip(UNIFAC(T, x_ls, UNIFAC_DG), x_ls)]

def fetch_UNIFAC_DG(full_name_list = ['isobutene', 'ethanol', 'ethyl tert-butyl ether'], update =False):
    UNIFAC_DG = []
    if update:
        for name in full_name_list:
            print(name)
            a = Chemical(name, 200, 100).UNIFAC_groups
            UNIFAC_DG.append(a)
        return UNIFAC_DG
    else:
        return [{1: 2, 7: 1}, {1: 1, 2: 1, 14: 1}, {1: 4, 4: 1, 25: 1}]

In [4]:
def Func(k =0.00047, Keq = 12.5872, B = 50, F= 7, D = 8):
    tspan = np.linspace(0, conc['t'][-1], 100)
    K1, K2 = 0,0
    def rate(arr):
        k2, Keq2 = k*20, Keq*20
        CIB, CEtOH, CETBE = activity(343, 101.325, 1, arr)
        return ((k*CIB*CEtOH - (k/Keq)*B*CETBE)/(CEtOH + B*CETBE + K1)) + ((k2*CIB*CEtOH - (k2/Keq2)*D*CETBE)/(CIB + F*CEtOH**2 + D*CETBE + K2))

    def batch(t, arr):
        r = rate(arr)
        return [-r, -r, r]
    y0 = [1.85, 1.8, 0]
    ans = solve_ivp(batch, [0, tspan[-1]], y0, dense_output=True).sol(tspan)
    plt.scatter(conc['t'], conc['IB'])
    plt.scatter(conc['t'], conc['EtOH'] )
    plt.scatter(conc['t'], conc['ETBE'])
    plt.plot(tspan, ans[0])
    plt.plot(tspan, ans[1])
    plt.plot(tspan, ans[2])
    plt.show()
    print('k ='+ str(k) + ', Keq = '+ str(Keq) + ', B = '+ str(B) + ', F= '+  str(F) +', D = ' +str(D))
#%%
interact(Func, k = (1e-6, 1, 1e-7), Keq = (1e-20, 20, 1e-5) , B=(0,50,10), F=(5,25,1), D=(0,20,2))

interactive(children=(FloatSlider(value=0.00047, description='k', max=1.0, min=1e-06, step=1e-07), FloatSlider…

<function __main__.Func(k=0.00047, Keq=12.5872, B=50, F=7, D=8)>

In [2]:
from thermo import Chemical, UNIFAC
def rate_french_ETBE(T, P, Q, arr, Ct = 4.7e-3): #Ct: mol/g cat, returns cat mass-based rate (mol/s.g)
    aIB, aEt, aw, aTBA, aETBE, aDIB, aTRIB, a1B, aIBane = activity(T, P , Q, arr)
    k, Keq = kf3(T), Ka(T)
    # a_sig = Ct/(B_Et(T)*aEt + B_ETBE(T)*aETBE)
    return k*(aIB*aEt*B_Et(T) - (aETBE/Keq))/(B_Et(T)*aEt + B_ETBE(T)*aETBE)
def activity(T, P, Q, arr): # takes concentration
    xIB, xEt, xw, xTBA, xETBE, xDIB, xTRIB, x1B, xIBane = x_ls = [C/sum(arr) for C in arr]
    names = ['isobutene', 'ethyl alcohol', 'water', 'tert-butanol', 'ethyl tert-butyl ether', '2,4,4-trimethyl-2-pentene', '4,4-dimethyl-2-neopentyl-1-pentene', '1-butene', 'isobutane']
    UNIFAC_DG = [{1: 2, 7: 1}, {1: 1, 2: 1, 14: 1}, {16: 1}, {1: 3, 4: 1, 14: 1}, {1: 4, 4: 1, 25: 1}, {1: 5, 4: 1, 8: 1}, {1: 6, 2: 2, 4: 2, 7: 1}, {1: 1, 2: 1, 5: 1}, {1: 3, 3: 1}]
    # for name in names:
    #     a = Chemical(name, T, P).UNIFAC_groups
    #     UNIFAC_DG.append(a)
    # return UNIFAC_DG
    return [g*x for g, x, in zip(UNIFAC(T, x_ls, UNIFAC_DG), x_ls)]

In [3]:
def B_Et(T, TrKr = [358, 1.05e4], dH = -8.3e3, rho_b = 640.26): # 323<T<358
    R = 8.314
    Tr, Kr = TrKr
    return Kr*np.exp((-dH/R)*((T**-1) - (Tr**-1)))/ rho_b

def B_IB(T, TrKr = [340, 260], dH = -54.2e3, rho_b = 640.26):
    R = 8.314
    Tr, Kr = TrKr
    return Kr*np.exp((-dH/R)*((T**-1) - (Tr**-1)))/ rho_b

def B_ETBE(T, Tr = [313, 323], Kadr = [7.3e-7, 3.23]):
    R = 8.314
    dH = -R*np.log(Kadr[1]/Kadr[0])/((Tr[1]**-1)-(Tr[0]**-1))
    Kr, Tr = Kadr[0], Tr[0]
    return Kr*np.exp((-dH/R)*((T**-1) - (Tr**-1)))

In [4]:
def MB(t, Cls):
    rETBE = rate_french_ETBE(T0, P0, Q0, Cls)*W*1000
    return [-rETBE, -rETBE, 0,0,rETBE,0,0,0,0]

In [14]:
def inte(Z1 = 1, Z2 = 1):
    T = 333
    Xer = 0.88
    def TAp(T, Z1, Z2):
        def BetaFunc(T):
            return 20*np.exp(Z1*((T**-1) - 343**-1))

        def FFunc(T):
            return 85*np.exp(Z2*((T**-1) - 343**-1))

        def Ka(T, Ka_r = 16.5, T_r =  343, dH = -44.3e3): # dH: kJ/kmol
            R = 8.314
            return Ka_r*np.exp((-dH/R)*((T**-1)- (T_r**-1)))

        def kf3(T, k3_r = 4.7e-4, Tr = 343, Ea = 81.2e3): #k3_r: dm3/g.s, Ea: kJ/kmol
            R = 8.314
            k0 = k3_r/np.exp(-Ea/(R*Tr))
            return k0*np.exp(-Ea/(R*T)) ## not verified yet...

        def rate_french_ETBE_conc(T, P, Q, arr, Ct = 4.7e-3): #Ct: mol/g cat, returns cat mass-based rate (mol/s.g)
            CIB, CEtOH, CETBE = arr
            k, Keq = kf3(T), Ka(T)
            α = 1/Keq
            F = FFunc(T)
            β = BetaFunc(T)
            return k*(CIB - α*(CETBE/CEtOH) + ((β*(CIB*CEtOH - α*CETBE))/(CIB + F*(CEtOH**2) + CETBE)))

        def batch(t, arr):
            r = rate_french_ETBE_conc(T, 1 , 1 , arr)
            return [-r, -r, r]

        y0 = [1.85, 1.8, 0]
        ans = solve_ivp(batch, [0, 150000], y0, dense_output=True)
        Xspan = []
        for CIB in ans['y'][0]:
            Xspan.append((ans['y'][0][0] - CIB)/ans['y'][0][0])
        return ans['t'], Xspan
    plt.plot(*TAp(T, Z1, Z2))

    plt.plot(*TAp(323, Z1, Z2))
    plt.plot(*TAp(343, Z1, Z2))
    plt.scatter(TAp(T, Z1, Z2)[0][-1], Xer)
    plt.scatter(TAp(323, Z1, Z2)[0][-1], 0.91)
#     plt.axis([199800, 200050, 0.8, 0.95])
interact(inte, Z1 = (-100000, 100000, 10000), Z2 = (-100000, 100000, 10000))

interactive(children=(IntSlider(value=1, description='Z1', max=100000, min=-100000, step=10000), IntSlider(val…

<function __main__.inte(Z1=1, Z2=1)>